In [ ]:
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.common.by import By
import re
import copy
from datetime import datetime
import time
import telegram_send
import requests

In [ ]:
hour_reg = re.compile("(0[0-9]|1[0-9]|2[0-3]):[0-5][0-9]")

In [ ]:
url = "https://www.olx.pl/d/nieruchomosci/stancje-pokoje/krakow/q-pok%C3%B3j/?search%5Border%5D=created_at:desc" 

driver = webdriver.Chrome(executable_path=r'C:\Users\Milosz\Desktop\chromedriver_win32\chromedriver.exe')
driver.get(url)

In [ ]:
cookie_handler = driver.find_element(By.ID, "onetrust-accept-btn-handler")
cookie_handler.click()

In [ ]:
# BEGIN:
offers = driver.find_elements(By.CLASS_NAME, "css-19ucd76")
dates = []
for o in offers: 
    try:
        date_details = o.find_element(By.CSS_SELECTOR, "p.css-p6wsjo-Text.eu5v0x0")
        dates.append(date_details.text)
    except Exception:
        pass

buffer = copy.copy(offers)
buffer_text = [b.text for b in buffer]

In [ ]:
telegram_group_id = '-627491859'
authentication_token = '5798349801:AAFSas7UHkLtgrkeb2-JN6J0KvYt1yHIlV4'

In [ ]:
while True:
    print("searching...")
    offers = driver.find_elements(By.CLASS_NAME, "css-19ucd76")
    for o in offers:
        try:
            if (o.text not in buffer_text) and (re.search(r"Dzisiaj", o.text)) and (not re.search(r'Odświeżono', o.text)):
                buffer.insert(0, o)
                buffer_text.insert(0, o.text)
                found = hour_reg.search(o.text)
                print(found.group())
                print(o.text, "CURRENT_TIME:", datetime.now())
                found_url = o.find_element(By.CSS_SELECTOR, "a.css-1bbgabe").get_attribute('href')
                #telegram_send.send(messages=[found.group(), found_url])
                message = f'{found.group()}, {found_url}'
                url_telegram = f'https://api.telegram.org/bot{authentication_token}/sendMessage?chat_id={telegram_group_id}&text={message}'
                out = requests.get(url_telegram)
        except Exception:
            pass
    time.sleep(60)
    driver.refresh()